In [ ]:
require 'redis'
require 'date'
require 'descriptive_statistics'
require 'pathname'


#vts(START_MONTH) and vte(END_MONTH) are included in interval: [vts,vte]
END_MONTH=[2017,12]
START_MONTH=[2017,1] #1 year version
#START_MONTH=[2017,1] #10 years version


redis = Redis.new(host: "redis", port: 6379, db: 0)

In [ ]:
class String

  def to_table_name
    
    self.gsub(/\W/, '')
  end

end

class Date
  def granule(granularity)
    granularities = [:year, :month, :day]
    idx =  granularities.index(granularity)
    granularities[0..idx].map{|g| eval "self.#{g}" }
  end
end

class Array

  def to_dot(filename = 'digraph.dot')
     File.open(filename, 'w') do |f|
       f.puts "digraph {"
       self.map{|e| f.puts "#{e[0]} -> #{e[1]};"}
       f.puts "}"
     end
  end

  def <(array)
    self.each_with_index do |i, idx|
      if i < array[idx] then  return true end
      if i > array[idx] then  return false end
    end
    return false
  end

   
  def devstd()
    m = self.mean
    sum = 0.0
    freq = Hash.new(0)
    self.each { |x| freq[x] += 1 }
    self.each {|v| sum += (v-m)**2 *freq[v]/freq.size }
    Math.sqrt(sum/self.size)
  end

          
end
      
      
def split(params = {})
  p = {verbose: false}.merge(params)
  series  = signal_series(p)
  total_sd = series.devstd
  total_mean = series.mean
  
  # epsilon=thr/n
  thr=5.0
  n= series.sum
  epsilon = thr/n
  
  #puts "n: #{n}"
  #puts "thr: #{thr}"
  #puts "epsilon: #{epsilon}"
  #puts "Series: #{series}"
  #puts "total sd: #{total_sd}"

  if p[:verbose] then puts "#{Time.now}: starting split drug: #{p[:drug]} adr:  #{p[:adr]} " end
  if p[:verbose]
    puts "#{Time.now}: series"
    series.each{|s| puts s}
  end
    
 edges = (0..series.size - 1).to_a.select { |i|
    series_values =  series[0..i].map{|j| j[1]}
    if series_values.mean > 0
      (series_values.devstd / series_values.mean) <= (total_sd / total_mean) * epsilon
     else 
       true
    end
  }.map{|i| [0,i + 1]}
  edges += (0..series.size - 1).to_a.select { |i|
    series_values =  series[i..series.size - 1].map{|j| j[1]}
   if series_values.mean > 0
      (series_values.devstd / series_values.mean) <= (total_sd / total_mean) * epsilon
     else 
       true
    end
  }.map{|i| [i + 1, series.size + 1]}
  edges +=  (0..series.size - 1).to_a.product((0..series.size - 1).to_a).select{|i| i[0] < i[1]}.select{|i|
    series_values =  series[i[0]..i[1] - 1].map{|j| j[1]}
    if series_values.mean > 0
      (series_values.devstd / series_values.mean) <= (total_sd / total_mean) * epsilon
     else 
       true
    end
  }.map{|i| [i[0] + 1, i[1] + 1 ]}
    

   shp(edges)
end

def shp(edges)
  t = edges.flatten.max
  shp = (0..t).to_a.map{|_| Float::INFINITY}; shp[0] = 0
  pre = (0..t).to_a.map{|_| nil}
  edges.sort{|a,b| a <=> b}.each{|e| if shp[e[1]] > shp[e[0]] + 1 then shp[e[1]] = shp[e[0]] + 1; pre[e[1]] = e[0] end}
  r = [t]; while !pre[r[-1]].nil? do  r.push(pre[r[-1]]) end
  r.reverse
end          

  
def init_temporal_values(params = {})
  (params[:start_date]..params[:end_date]).to_a.map{|ts| ts.granule(params[:granularity]) }.uniq
end  
  
  
SOC_REDIS = redis  

def signal_series(params = {})
  p = {
       redis: SOC_REDIS,
       adr_level: :soc,
       granularity: :month,
       vts:  START_MONTH,
       vte: END_MONTH
  }.merge(params)
  
  # drug, adr, is_soc, granularity  = p[:drug], p[:adr], p[:soc?], p[:granularity]
  tv = init_temporal_values(start_date: Date.new(p[:vts][0],p[:vts][1],1), 
                       end_date: Date.new(p[:vte][0],p[:vte][1],1), 
                       granularity: :month)
  r = tv.map do |y,m|
                          key = {incremental: false, year: y, month: m, drug: p[:drug], p[:adr_level] => p[:adr] }  
                          v =  p[:redis].get(key)
                          v.nil? ? 0 :  v.to_i
                       end
  p[:granularity] == :year ?  r.each_slice(12).to_a.map{|i| i.reduce(:+)} : r
  
end

In [ ]:
#for each pair in redis db obtain a set of homogeneous intervals and write to file 

#Obtain all drug-adr pairs
pairs = redis.keys({incremental: true,year: END_MONTH[0], month: END_MONTH[1], drug: "*", soc: "*"}.to_s); nil
pairs = pairs.map{|p| h = (eval p); {drug: h[:drug], adr: h[:soc]} }.uniq

Pathname('results/TEDAR/split/split_TEDAR.txt').dirname.mkpath

#for each drug-adr pair compute TEDAR extraction of homogenous intervals and write intervals to file
pairs.each{|p| 
  split=split(p.merge({granularity: :month, verbose: false}))
  s= split.map { |i| i.to_s }.join(",")
  File.write("results/TEDAR/split/split_TEDAR.txt", p[:drug]+"\t"+p[:adr]+"\t"+s+"\n", mode: "a")
}